In [1]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path
import numpy as np
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam

Using TensorFlow backend.


In [8]:
dmm = pd.read_csv("FinalWithDecade.csv")

In [9]:
song_year = np.asarray(dmm['Year'])

In [10]:
# because some other features numbers are too low or to high outside of the range (0,1) so I am afraid that the model won't be able
# to predict
features= np.asarray(dmm[['Danceability',
        'Energy',
        'Speechness',
        'Acousticness',
        'Instrumentalness',
        'Liveness',
        'Valence']]).tolist()

In [11]:
decade_label= []
for el in song_year:
    e = int(el)
    if e<1970 and (e >1960 or e ==1960):
        decade_label.append('1960s')
    elif e<1980 and (e >1970 or e ==1970):
        decade_label.append('1970s')
    elif e<1990 and (e >1980 or e ==1980):
        decade_label.append('1980s')
    elif e<2000 and (e >1990 or e ==1990):
        decade_label.append('1990s')
    elif e<2010 and (e >2000 or e ==2000):
        decade_label.append('2000s')
    elif e<2020 and (e >2010 or e ==2010):
        decade_label.append('2010s')

In [12]:
print(len(decade_label))

5717


In [13]:
print(np.asarray(features))

[['0.466' '0.389' '0.0253' ... '0.843' '0.295' '0.749']
 ['0.554' '0.186' '0.0379' ... '0.00144' '0.11' '0.2']
 ['0.498' '0.582' '0.0339' ... '0' '0.372' '0.866']
 ...
 ['0.894' '0.511' '0.42' ... '0' '0.14' '0.568']
 ['0.606' '0.652' '0.0281' ... '0' '0.201' '0.476']
 ['0.575' '0.537' '0.031' ... '0.825' '0.0903' '0.342']]


In [14]:
song_title = np.asarray(dmm['Title'])

for e in features:
    if 'None' in e:
        print(features.index(e))

5245
5245


# We have to remove 2 songs that do not have analysis. 

In [16]:
f1 = features.pop(5245)

In [17]:
features.pop(5390)

['None', 'None', 'None', 'None', 'None', 'None', 'None']

In [18]:

decade_label.pop(5245)
decade_label.pop(5390)
print(len(decade_label))

5715


In [19]:
# Because the values in the csv files are string, so we have to convert them back to double values.
d_features = []
for e in features:
    d_features.append([np.double(i) for i in e])
print(len(d_features))
#print(d_features)

5715


In [20]:
decade2 = []
for e in decade_label:
    if '1960s' in e:
        decade2.append(0)
    elif '1970s' in e:
        decade2.append(0)
    elif '1980s' in e:
        decade2.append(0)
    elif '1990s' in e:
        decade2.append(0)
    elif '2000s' in e:
        decade2.append(1)
    elif '2010s' in e:
        decade2.append(1)
print(decade2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [285]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
model = Sequential()
model.add(Dense(4000, activation='relu', input_dim = 7))

model.add(Dense(2000, activation='relu'))

model.add(Dense(2000, activation='relu'))


model.add(Dense(2000, activation='relu'))

model.add(Dense(2000, activation='relu'))

model.add(Dense(1000, activation='relu'))

model.add(Dense(1000, activation='relu'))

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])




In [286]:
model.fit(np.asarray(d_features),np.asarray(decade2), epochs=400, batch_size=128)

Epoch 1/400
5715/5715 [==============================] - 14s 3ms/step - loss: 0.5938 - accuracy: 0.7118
Epoch 2/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4858 - accuracy: 0.7680
Epoch 3/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4734 - accuracy: 0.7815
Epoch 4/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4603 - accuracy: 0.7808
Epoch 5/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4562 - accuracy: 0.7799
Epoch 6/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4495 - accuracy: 0.7834
Epoch 7/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4527 - accuracy: 0.7879
Epoch 8/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4539 - accuracy: 0.7818
Epoch 9/400
5715/5715 [==============================] - 15s 3ms/step - loss: 0.4345 - accuracy: 0.7902
Epoch 10/400
5715/5715 [==============================] - 15s 3m

5715/5715 [==============================] - 17s 3ms/step - loss: 0.3672 - accuracy: 0.8248
Epoch 80/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3712 - accuracy: 0.8185
Epoch 81/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3653 - accuracy: 0.8238
Epoch 82/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.3614 - accuracy: 0.8234
Epoch 83/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3656 - accuracy: 0.8275
Epoch 84/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3682 - accuracy: 0.8212
Epoch 85/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3685 - accuracy: 0.8276
Epoch 86/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3687 - accuracy: 0.8206
Epoch 87/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.3717 - accuracy: 0.8231
Epoch 88/400
5715/5715 [==============================] - 17s 3ms/st

5715/5715 [==============================] - 18s 3ms/step - loss: 0.0614 - accuracy: 0.9774
Epoch 234/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.0969 - accuracy: 0.9650
Epoch 235/400
5715/5715 [==============================] - 19s 3ms/step - loss: 0.0714 - accuracy: 0.9713
Epoch 236/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0666 - accuracy: 0.9762
Epoch 237/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0544 - accuracy: 0.9804
Epoch 238/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.0459 - accuracy: 0.9808
Epoch 239/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.0572 - accuracy: 0.9780
Epoch 240/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.0544 - accuracy: 0.9795
Epoch 241/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.0398 - accuracy: 0.9848
Epoch 242/400
5715/5715 [==============================] - 1

5715/5715 [==============================] - 17s 3ms/step - loss: 0.0289 - accuracy: 0.9906
Epoch 388/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0223 - accuracy: 0.9939
Epoch 389/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0220 - accuracy: 0.9925
Epoch 390/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0341 - accuracy: 0.9900
Epoch 391/400
5715/5715 [==============================] - 18s 3ms/step - loss: 0.0273 - accuracy: 0.9920
Epoch 392/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0120 - accuracy: 0.9960
Epoch 393/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0260 - accuracy: 0.9916
Epoch 394/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0213 - accuracy: 0.9920
Epoch 395/400
5715/5715 [==============================] - 17s 3ms/step - loss: 0.0239 - accuracy: 0.9928
Epoch 396/400
5715/5715 [==============================] - 1

In [287]:
model.save("modelbig.h5")

In [17]:
score = model.evaluate(np.asarray(d_features),np.asarray(decade2), batch_size=128)
print(score)

5715/5715 [==============================] - 2s 265us/step
[0.049211214768943926, 0.9847769141197205]


In [268]:
data = np.random.random((1000, 100))
data

array([[0.0256551 , 0.14911294, 0.48078962, ..., 0.39456541, 0.75862309,
        0.65406684],
       [0.23056592, 0.7761442 , 0.80140953, ..., 0.71214484, 0.1826715 ,
        0.23377635],
       [0.19853305, 0.32990571, 0.08362414, ..., 0.52562893, 0.50223986,
        0.0518215 ],
       ...,
       [0.05987745, 0.13415019, 0.95717477, ..., 0.47130554, 0.5519935 ,
        0.21694546],
       [0.5680946 , 0.85994242, 0.47109544, ..., 0.37219598, 0.8400485 ,
        0.48982125],
       [0.51123782, 0.93862447, 0.53514196, ..., 0.13842624, 0.42750163,
        0.7622708 ]])

In [2]:
from numpy import loadtxt
from keras.models import load_model
model=load_model("modelbig.h5")

In [27]:
a = [ 0.687, 0.514,0.0897, 0.366, 0 ,0.116, 0.494]
x_test = np.asarray(a)
s = model.predict(np.asarray(d_features)).tolist()
print(s)

[[3.772394476106703e-36], [0.01859370991587639], [6.274290154217592e-22], [0.0024255511816591024], [1.2386394701024983e-06], [7.834437265596961e-11], [3.087945543711612e-08], [0.1142893135547638], [0.019203608855605125], [1.6260250253457426e-13], [0.004224542994052172], [0.012010501697659492], [0.0008697481243871152], [0.005894571542739868], [1.7245546755331755e-35], [0.0012663460802286863], [0.04431961104273796], [7.68006793805398e-06], [1.0003718671214301e-05], [0.003442514454945922], [0.0030464690644294024], [5.284687262314047e-16], [1.5785487903485773e-06], [8.859118771908925e-09], [2.6719604787610088e-27], [0.009070423431694508], [0.0007676858804188669], [1.947547352455016e-26], [0.001627274788916111], [1.7054355339496396e-05], [0.0012447964400053024], [9.156234888667934e-31], [1.2439369162281413e-14], [1.573854809227681e-14], [2.5777274004212813e-06], [0.003592639695852995], [1.9075283616120514e-13], [0.0], [3.118303197879868e-07], [4.416073694332896e-14], [5.105238605994378e-13]

In [57]:
b = [0.743, 0.752, 0.0414, 0.0188, 0, 0.287, 0.545]
x_test=[b]

In [58]:
model.predict(np.asarray(x_test))

array([[0.9995297]], dtype=float32)

In [19]:
print(decade2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Testing the model on Justin Bieber and he Beatles tracks

In [4]:
justin_dic = {'All Around Me': [0.566, 0.438, 0.0367, 0.809, 0.0, 0.212, 0.692], 'Habitual': [0.764, 0.443, 0.0439, 0.52, 1.96e-05, 0.0749, 0.63], 'Come Around Me': [0.648, 0.681, 0.0378, 0.263, 0.00783, 0.0971, 0.767], 'Intentions (feat. Quavo)': [0.806, 0.546, 0.0575, 0.3, 0.0, 0.102, 0.874], 'Yummy': [0.676, 0.506, 0.0958, 0.345, 0.0, 0.118, 0.497], 'Available': [0.702, 0.608, 0.0359, 0.0736, 0.0, 0.129, 0.597], 'Forever (feat. Post Malone & Clever)': [0.825, 0.483, 0.08, 0.538, 1.48e-05, 0.204, 0.892], 'Running Over (feat. Lil Dicky)': [0.774, 0.603, 0.0591, 0.438, 4.37e-05, 0.0869, 0.977], 'Take It Out On Me': [0.772, 0.616, 0.102, 0.00337, 2.36e-06, 0.104, 0.279], 'Second Emotion (feat. Travis Scott)': [0.796, 0.777, 0.0567, 0.0276, 2.87e-05, 0.095, 0.96], 'Get Me (feat. Kehlani)': [0.832, 0.508, 0.149, 0.0492, 5.39e-05, 0.107, 0.75], 'E.T.A.': [0.465, 0.494, 0.066, 0.49, 0.0, 0.127, 0.457], 'Changes': [0.485, 0.412, 0.042, 0.738, 0.0, 0.12, 0.612], 'Confirmation': [0.438, 0.315, 0.0375, 0.608, 0.0, 0.151, 0.449], "That's What Love Is": [0.639, 0.437, 0.0355, 0.692, 0.0, 0.12, 0.751], 'At Least For Now': [0.375, 0.304, 0.0325, 0.683, 0.0, 0.196, 0.295], 'Yummy - Summer Walker Remix': [0.655, 0.578, 0.0534, 0.311, 0.0, 0.109, 0.546], 'Mark My Words': [0.383, 0.256, 0.0332, 0.88, 2.69e-06, 0.169, 0.519], "I'll Show You": [0.361, 0.608, 0.098, 0.05, 7.96e-05, 0.183, 0.0789], 'What Do You Mean?': [0.845, 0.567, 0.0956, 0.59, 0.00142, 0.0811, 0.793], 'Sorry': [0.654, 0.76, 0.045, 0.0797, 0.0, 0.299, 0.41], 'Love Yourself': [0.609, 0.378, 0.438, 0.835, 0.0, 0.28, 0.515], 'Company': [0.59, 0.797, 0.0614, 0.132, 0.0, 0.0846, 0.425], 'No Pressure': [0.644, 0.595, 0.215, 0.121, 0.0, 0.279, 0.554], 'No Sense': [0.643, 0.499, 0.0308, 0.0478, 0.0, 0.108, 0.363], 'The Feeling': [0.525, 0.809, 0.0619, 0.0819, 1.22e-06, 0.0971, 0.21], 'Life Is Worth Living': [0.558, 0.296, 0.0427, 0.907, 0.0, 0.108, 0.463], 'Where Are Ü Now': [0.432, 0.74, 0.062, 0.0342, 6.59e-06, 0.14, 0.193], 'Children': [0.634, 0.885, 0.304, 0.0122, 0.0, 0.193, 0.327], 'Purpose': [0.481, 0.285, 0.0424, 0.9, 0.0, 0.123, 0.316], 'Been You': [0.74, 0.781, 0.07, 0.0223, 0.0, 0.321, 0.861], 'Get Used To It': [0.687, 0.705, 0.0522, 0.0339, 0.0, 0.164, 0.449], 'We Are': [0.586, 0.774, 0.261, 0.255, 0.0, 0.137, 0.651], 'Trust': [0.512, 0.494, 0.206, 0.0948, 0.0, 0.124, 0.492], 'All In It': [0.459, 0.701, 0.0782, 0.502, 3.62e-05, 0.238, 0.509], 'What Do You Mean? - Acoustic': [0.773, 0.358, 0.044, 0.794, 0.0, 0.104, 0.756], 'Heartbreaker': [0.572, 0.503, 0.0858, 0.156, 0.0, 0.346, 0.553], 'All That Matters': [0.696, 0.513, 0.0424, 0.00508, 0.0, 0.0682, 0.144], 'Hold Tight': [0.456, 0.608, 0.328, 0.287, 0.0, 0.101, 0.491], 'Recovery': [0.509, 0.522, 0.189, 0.013, 0.0, 0.329, 0.229], 'Bad Day': [0.663, 0.506, 0.0431, 0.0186, 0.0, 0.237, 0.323], 'All Bad': [0.694, 0.413, 0.0259, 0.0197, 0.000905, 0.195, 0.36], 'PYD': [0.412, 0.494, 0.0394, 0.837, 0.0, 0.431, 0.421], 'Roller Coaster': [0.706, 0.864, 0.0677, 0.0119, 0.0, 0.205, 0.576], 'Change Me': [0.303, 0.347, 0.0932, 0.579, 0.0, 0.19, 0.0829], 'Confident': [0.738, 0.563, 0.11, 0.113, 0.0, 0.191, 0.876], 'One Life': [0.371, 0.557, 0.139, 0.301, 0.0, 0.111, 0.464], 'Backpack': [0.543, 0.76, 0.131, 0.152, 0.0, 0.206, 0.498], "What's Hatnin'": [0.57, 0.756, 0.0829, 0.0234, 0.0, 0.419, 0.37], 'Swap It Out': [0.426, 0.662, 0.055, 0.0318, 0.0, 0.0811, 0.199], 'Memphis': [0.639, 0.727, 0.0496, 0.0114, 0.0, 0.0752, 0.505], 'Flatline': [0.341, 0.666, 0.065, 0.483, 0.0, 0.194, 0.359], 'Boyfriend - Acoustic Version': [0.752, 0.292, 0.0924, 0.842, 0.0, 0.1, 0.62], 'As Long As You Love Me - Acoustic Version': [0.554, 0.307, 0.0351, 0.776, 0.0, 0.0858, 0.428], 'Beauty And A Beat - Acoustic Version': [0.69, 0.302, 0.0326, 0.734, 0.0, 0.13, 0.699], "She Don't Like The Lights - Acoustic Version": [0.612, 0.428, 0.0292, 0.75, 0.0, 0.0921, 0.685], 'Take You - Acoustic Version': [0.57, 0.341, 0.0345, 0.788, 0.0, 0.17, 0.715], 'Be Alright - Acoustic Version': [0.315, 0.253, 0.0347, 0.916, 0.0, 0.208, 0.457], 'All Around The World - Acoustic Version': [0.591, 0.345, 0.0435, 0.621, 0.0, 0.214, 0.715], 'Fall - Live': [0.4, 0.366, 0.0398, 0.828, 0.0, 0.111, 0.625], 'Yellow Raincoat': [0.783, 0.375, 0.0328, 0.708, 0.121, 0.0933, 0.742], 'I Would': [0.707, 0.647, 0.0881, 0.456, 0.0, 0.204, 0.578], 'Nothing Like Us - Bonus Track': [0.529, 0.155, 0.0428, 0.987, 6.02e-05, 0.18, 0.511], 'All Around The World': [0.689, 0.816, 0.178, 0.019, 2.42e-06, 0.181, 0.775], 'Boyfriend': [0.717, 0.55, 0.0521, 0.0358, 0.00196, 0.126, 0.332], 'As Long As You Love Me': [0.571, 0.873, 0.0997, 0.0807, 0.0, 0.361, 0.613], 'Catching Feelings': [0.327, 0.542, 0.0412, 0.354, 0.0, 0.35, 0.357], 'Take You': [0.663, 0.706, 0.0348, 0.00107, 0.00248, 0.232, 0.461], 'Right Here': [0.639, 0.57, 0.042, 0.0233, 0.0, 0.329, 0.728], 'Fall': [0.607, 0.754, 0.0411, 0.259, 0.0, 0.111, 0.607], 'Die In Your Arms': [0.568, 0.878, 0.0933, 0.0309, 4.26e-06, 0.213, 0.725], 'Thought Of You': [0.697, 0.705, 0.031, 0.0094, 6.61e-06, 0.105, 0.57], 'Beauty And A Beat': [0.601, 0.843, 0.0593, 0.000704, 5.27e-05, 0.0681, 0.534], 'One Love': [0.688, 0.612, 0.0324, 0.00276, 0.0, 0.0942, 0.228], 'Be Alright': [0.803, 0.356, 0.0326, 0.587, 0.0, 0.16, 0.738], 'Believe': [0.519, 0.854, 0.113, 0.143, 0.0, 0.171, 0.508], 'Out Of Town Girl': [0.594, 0.879, 0.0715, 0.126, 6.32e-06, 0.151, 0.799], "She Don't Like The Lights": [0.575, 0.896, 0.103, 0.00579, 0.00217, 0.318, 0.44], 'Maria': [0.825, 0.689, 0.0413, 0.01, 1.37e-05, 0.19, 0.3], 'Fairytale': [0.548, 0.805, 0.0754, 0.0271, 0.601, 0.11, 0.814], 'Never Say Never': [0.739, 0.807, 0.0647, 0.0181, 0.0, 0.295, 0.633], 'That Should Be Me': [0.547, 0.607, 0.0319, 0.21, 0.0, 0.169, 0.23], 'Somebody To Love Remix': [0.704, 0.836, 0.0283, 0.00388, 0.0, 0.16, 0.553], 'Up': [0.71, 0.712, 0.0383, 0.0766, 0.0, 0.105, 0.552], 'Overboard (Live)': [0.357, 0.884, 0.0699, 0.425, 0.0, 0.791, 0.384], 'Runaway Love - Kanye West Remix': [0.742, 0.742, 0.0929, 0.14, 0.0, 0.0729, 0.819], 'Born To Be Somebody': [0.674, 0.612, 0.0293, 0.194, 0.0, 0.0493, 0.31], 'Only Thing I Ever Get For Christmas': [0.478, 0.63, 0.0518, 0.217, 0.0, 0.111, 0.324], 'Mistletoe': [0.658, 0.56, 0.0467, 0.498, 0.0, 0.0794, 0.854], 'The Christmas Song (Chestnuts Roasting On An Open Fire)': [0.265, 0.436, 0.0411, 0.45, 0.0, 0.102, 0.159], 'Santa Claus Is Coming To Town': [0.647, 0.758, 0.0599, 0.0272, 0.0, 0.292, 0.838], 'Fa La La': [0.657, 0.679, 0.0361, 0.0207, 3.15e-06, 0.198, 0.416], 'All I Want For Christmas Is You (SuperFestive!) Duet with Mariah Carey': [0.576, 0.911, 0.064, 0.306, 0.0, 0.434, 0.401], 'Drummer Boy': [0.751, 0.848, 0.0823, 0.0248, 0.000127, 0.149, 0.479], 'Christmas Eve': [0.411, 0.57, 0.0707, 0.689, 0.0, 0.173, 0.335], 'All I Want Is You': [0.564, 0.823, 0.0405, 0.0461, 0.0, 0.109, 0.482], 'Home This Christmas': [0.539, 0.628, 0.0312, 0.271, 0.0, 0.112, 0.493], 'Silent Night': [0.286, 0.0346, 0.0404, 0.958, 0.0, 0.115, 0.172], 'All I Want For Christmas Is You (SuperFestive!)': [0.576, 0.911, 0.064, 0.306, 0.0, 0.434, 0.401], 'Christmas Love': [0.633, 0.512, 0.0294, 0.518, 0.0, 0.328, 0.376], 'Fa La La - (a cappella)': [0.61, 0.207, 0.0319, 0.847, 0.0, 0.152, 0.687], 'Pray': [0.57, 0.797, 0.0599, 0.124, 1.13e-06, 0.105, 0.664], 'Someday At Christmas': [0.523, 0.465, 0.0328, 0.439, 0.0, 0.22, 0.263], 'All I Want Is You - Acoustic': [0.504, 0.481, 0.0329, 0.67, 0.0, 0.163, 0.581], 'Baby': [0.73, 0.848, 0.101, 0.0494, 0.0, 0.133, 0.559], 'Somebody To Love': [0.719, 0.839, 0.0257, 0.00241, 0.0, 0.228, 0.636], 'Stuck In The Moment': [0.718, 0.709, 0.0465, 0.15, 0.0, 0.156, 0.452], 'U Smile': [0.705, 0.605, 0.03, 0.519, 1.56e-06, 0.117, 0.419], 'Runaway Love': [0.743, 0.633, 0.0319, 0.0816, 0.0, 0.0966, 0.906], 'Never Let You Go': [0.66, 0.802, 0.0373, 0.00205, 0.0, 0.399, 0.62], 'Overboard': [0.48, 0.79, 0.0501, 0.203, 0.0, 0.118, 0.468], 'Eenie Meenie': [0.718, 0.605, 0.0307, 0.0443, 0.0, 0.0962, 0.818], 'One Time': [0.691, 0.853, 0.0372, 0.0631, 7.13e-05, 0.082, 0.762], 'Favorite Girl': [0.581, 0.681, 0.0435, 0.0472, 0.0, 0.104, 0.634], 'Down To Earth': [0.543, 0.718, 0.0268, 0.432, 0.0, 0.437, 0.155], 'Bigger': [0.78, 0.733, 0.112, 0.166, 0.0, 0.341, 0.895], 'One Less Lonely Girl': [0.58, 0.75, 0.194, 0.159, 0.0, 0.274, 0.728], 'First Dance': [0.646, 0.52, 0.0412, 0.0137, 0.0, 0.275, 0.328], 'Love Me': [0.729, 0.868, 0.0609, 0.00939, 0.0, 0.283, 0.632], 'Common Denominator - iTunes Bonus': [0.655, 0.66, 0.0274, 0.227, 0.0, 0.0549, 0.45], 'Where Are You Now?': [0.574, 0.481, 0.0263, 0.00606, 3.46e-05, 0.146, 0.0393]}
beatles_dic = {'Come Together - 2019 Mix': [0.536, 0.36, 0.0408, 0.0823, 0.167, 0.0996, 0.147], 'Something - 2019 Mix': [0.416, 0.385, 0.0279, 0.0958, 5.24e-06, 0.228, 0.458], "Maxwell's Silver Hammer - 2019 Mix": [0.816, 0.386, 0.0382, 0.497, 9.31e-05, 0.309, 0.708], 'Oh! Darling - 2019 Mix': [0.437, 0.669, 0.0395, 0.0344, 0.00395, 0.347, 0.514], "Octopus's Garden - 2019 Mix": [0.565, 0.645, 0.0332, 0.113, 0.000625, 0.124, 0.626], "I Want You (She's So Heavy) - 2019 Mix": [0.326, 0.5, 0.0315, 0.0208, 0.0473, 0.0941, 0.365], 'Here Comes The Sun - 2019 Mix': [0.569, 0.557, 0.0428, 0.038, 0.000228, 0.136, 0.292], 'Because - 2019 Mix': [0.178, 0.283, 0.0298, 0.613, 8.04e-05, 0.127, 0.19], 'You Never Give Me Your Money - 2019 Mix': [0.33, 0.442, 0.035, 0.317, 0.0762, 0.149, 0.326], 'Sun King - 2019 Mix': [0.406, 0.274, 0.0301, 0.186, 0.138, 0.258, 0.199], 'Mean Mr Mustard - 2019 Mix': [0.69, 0.674, 0.0391, 0.0254, 6.59e-06, 0.47, 0.915], 'Polythene Pam - 2019 Mix': [0.47, 0.833, 0.0569, 0.00203, 0.706, 0.133, 0.491], 'She Came In Through The Bathroom Window - 2019 Mix': [0.527, 0.752, 0.0298, 0.00307, 4.49e-05, 0.329, 0.738], 'Golden Slumbers - 2019 Mix': [0.395, 0.181, 0.0284, 0.303, 0.000184, 0.16, 0.221], 'Carry That Weight - 2019 Mix': [0.494, 0.558, 0.0283, 0.0915, 0.000786, 0.243, 0.625], 'The End - 2019 Mix': [0.624, 0.532, 0.0606, 0.00045, 0.142, 0.151, 0.125], 'Her Majesty - 2019 Mix': [0.742, 0.182, 0.0986, 0.647, 2.9e-06, 0.344, 0.565], "I Want You (She's So Heavy) - Trident Recording Session & Reduction Mix": [0.36, 0.383, 0.0384, 0.13, 0.00115, 0.273, 0.373], 'Goodbye - Home Demo': [0.556, 0.174, 0.0462, 0.918, 0.000681, 0.114, 0.563], 'Something - Studio Demo': [0.408, 0.187, 0.035, 0.955, 0.000575, 0.0766, 0.243], 'The Ballad Of John And Yoko - Take 7': [0.695, 0.732, 0.0545, 0.0681, 0.0, 0.164, 0.672], 'Old Brown Shoe - Take 2': [0.639, 0.731, 0.0504, 0.447, 0.0, 0.0371, 0.372], 'Oh! Darling - Take 4': [0.596, 0.51, 0.0285, 0.307, 2.95e-05, 0.326, 0.545], "Octopus's Garden - Take 9": [0.699, 0.412, 0.098, 0.0851, 1.44e-05, 0.178, 0.837], 'You Never Give Me Your Money - Take 36': [0.375, 0.455, 0.0635, 0.621, 0.00568, 0.131, 0.377], 'Her Majesty - Takes 1-3': [0.691, 0.134, 0.355, 0.75, 0.00153, 0.144, 0.509], 'Golden Slumbers / Carry That Weight - Takes 1-3 / Medley': [0.561, 0.202, 0.0331, 0.674, 0.00102, 0.147, 0.235], 'Here Comes The Sun - Take 9': [0.581, 0.555, 0.0305, 0.0408, 0.00492, 0.0917, 0.562], "Maxwell's Silver Hammer - Take 12": [0.787, 0.319, 0.146, 0.237, 0.000506, 0.251, 0.701], 'Come Together - Take 5': [0.598, 0.314, 0.171, 0.096, 0.000621, 0.149, 0.277], 'The End - Take 3': [0.671, 0.42, 0.21, 0.000927, 0.706, 0.507, 0.539], 'Come And Get It - Studio Demo': [0.572, 0.405, 0.0344, 0.00356, 0.0, 0.178, 0.494], 'Sun King - Take 20': [0.559, 0.23, 0.0621, 0.444, 0.0305, 0.182, 0.286], 'Mean Mr. Mustard - Take 20': [0.59, 0.553, 0.0421, 0.023, 0.0152, 0.538, 0.628], 'Polythene Pam - Take 27': [0.479, 0.708, 0.0871, 0.0734, 0.0252, 0.371, 0.518], 'She Came In Through The Bathroom Window - Take 27': [0.594, 0.681, 0.0286, 0.00673, 0.00128, 0.1, 0.858], 'Because - Take 1 / Instrumental': [0.328, 0.381, 0.0745, 0.458, 0.191, 0.0799, 0.2], 'The Long One - Comprising of ‘You Never Give Me Your Money’, ’Sun King’/’Mean Mr Mustard’, ‘Her Majesty’, ‘Polythene Pam’/’She Came In Through The Bathroom Window’, ’Golden Slumbers’/ ’Carry That Weight’, ’The End’': [0.403, 0.425, 0.0405, 0.0565, 0.00839, 0.0788, 0.243], 'Something - Take 39 / Instrumental / Strings Only': [0.0959, 0.229, 0.0388, 0.79, 0.942, 0.0715, 0.0677], 'Golden Slumbers / Carry That Weight - Take 17 / Instrumental / Strings & Brass Only': [0.178, 0.218, 0.0336, 0.829, 0.768, 0.101, 0.15], 'Back In The U.S.S.R. - 2018 Mix': [0.51, 0.893, 0.0342, 0.00102, 5.85e-05, 0.201, 0.934], 'Dear Prudence - 2018 Mix': [0.395, 0.538, 0.0456, 0.0904, 0.000453, 0.0685, 0.369], 'Glass Onion - 2018 Mix': [0.529, 0.555, 0.0508, 0.00236, 0.003, 0.093, 0.615], 'Ob-La-Di, Ob-La-Da - 2018 Mix': [0.803, 0.776, 0.0333, 0.149, 0.00471, 0.222, 0.969], 'Wild Honey Pie - 2018 Mix': [0.749, 0.689, 0.0346, 0.278, 0.759, 0.796, 0.299], 'The Continuing Story Of Bungalow Bill - 2018 Mix': [0.659, 0.56, 0.0648, 0.295, 0.0, 0.683, 0.738], 'While My Guitar Gently Weeps - 2018 Mix': [0.421, 0.668, 0.0364, 0.00196, 0.0172, 0.307, 0.635], 'Happiness Is A Warm Gun - 2018 Mix': [0.461, 0.534, 0.0592, 0.131, 5.63e-06, 0.107, 0.459], 'Martha My Dear - 2018 Mix': [0.641, 0.328, 0.0313, 0.788, 7.77e-05, 0.157, 0.858], "I'm So Tired - 2018 Mix": [0.44, 0.485, 0.28, 0.496, 1.11e-05, 0.508, 0.717], 'Blackbird - 2018 Mix': [0.649, 0.123, 0.0396, 0.825, 3.14e-06, 0.0627, 0.358], 'Piggies - 2018 Mix': [0.527, 0.471, 0.0314, 0.2, 0.0, 0.0981, 0.571], 'Rocky Raccoon - 2018 Mix': [0.543, 0.429, 0.0397, 0.424, 0.0, 0.25, 0.638], "Don't Pass Me By - 2018 Mix": [0.598, 0.58, 0.0262, 0.472, 1.07e-06, 0.248, 0.961], "Why Don't We Do It In The Road? - 2018 Mix": [0.532, 0.524, 0.0545, 0.493, 8.68e-06, 0.0601, 0.845], 'I Will - 2018 Mix': [0.621, 0.348, 0.0251, 0.661, 0.000294, 0.0899, 0.9], 'Julia - 2018 Mix': [0.399, 0.091, 0.0314, 0.849, 0.000105, 0.106, 0.185], 'Birthday - 2018 Mix': [0.587, 0.662, 0.0398, 2.27e-05, 0.165, 0.194, 0.964], 'Yer Blues - 2018 Mix': [0.328, 0.608, 0.0467, 0.000652, 0.216, 0.0948, 0.48], "Mother Nature's Son - 2018 Mix": [0.564, 0.197, 0.0255, 0.614, 0.208, 0.271, 0.473], "Everybody's Got Something To Hide Except Me And My Monkey - 2018 Mix": [0.503, 0.775, 0.0683, 0.00445, 5.31e-05, 0.0554, 0.738], 'Sexy Sadie - 2018 Mix': [0.483, 0.546, 0.0299, 0.279, 0.00113, 0.198, 0.732], 'Helter Skelter - 2018 Mix': [0.22, 0.884, 0.0913, 0.000218, 0.141, 0.828, 0.248], 'Long, Long, Long - 2018 Mix': [0.337, 0.23, 0.0282, 0.35, 0.139, 0.103, 0.106], 'Revolution 1 - 2018 Mix': [0.37, 0.674, 0.0411, 0.0974, 0.000296, 0.427, 0.583], 'Honey Pie - 2018 Mix': [0.592, 0.357, 0.0622, 0.893, 4.96e-05, 0.136, 0.489], 'Savoy Truffle - 2018 Mix': [0.63, 0.616, 0.0504, 0.00938, 0.00962, 0.0792, 0.912], 'Cry Baby Cry - 2018 Mix': [0.481, 0.475, 0.0264, 0.129, 0.0209, 0.196, 0.489], 'Revolution 9 - 2018 Mix': [0.193, 0.706, 0.369, 0.775, 0.0908, 0.782, 0.119], 'Good Night - 2018 Mix': [0.131, 0.343, 0.0349, 0.888, 0.383, 0.107, 0.137], 'Back In The U.S.S.R. - Esher Demo': [0.658, 0.53, 0.0845, 0.756, 0.0, 0.185, 0.557], 'Dear Prudence - Esher Demo': [0.387, 0.266, 0.0302, 0.655, 0.0129, 0.143, 0.271], 'Glass Onion - Esher Demo': [0.697, 0.288, 0.0441, 0.689, 0.0, 0.127, 0.616], 'Ob-La-Di, Ob-La-Da - Esher Demo': [0.76, 0.601, 0.0495, 0.403, 0.0, 0.0859, 0.549], 'The Continuing Story Of Bungalow Bill - Esher Demo': [0.594, 0.357, 0.0397, 0.642, 0.0, 0.163, 0.494], 'While My Guitar Gently Weeps - Esher Demo': [0.503, 0.321, 0.0468, 0.835, 0.0, 0.353, 0.36], 'Happiness Is A Warm Gun - Esher Demo': [0.587, 0.18, 0.0424, 0.771, 2.41e-05, 0.161, 0.186], "I'm So Tired - Esher Demo": [0.473, 0.285, 0.101, 0.756, 0.0, 0.925, 0.42], 'Blackbird - Esher Demo': [0.585, 0.272, 0.0342, 0.699, 0.000274, 0.0758, 0.388], 'Piggies - Esher Demo': [0.447, 0.225, 0.0445, 0.868, 0.0, 0.341, 0.464], 'Rocky Raccoon - Esher Demo': [0.6, 0.297, 0.0349, 0.619, 0.0, 0.301, 0.356], 'Julia - Esher Demo': [0.507, 0.278, 0.0306, 0.845, 0.0344, 0.228, 0.261], 'Yer Blues - Esher Demo': [0.567, 0.467, 0.0432, 0.374, 0.000416, 0.511, 0.643], "Mother Nature's Son - Esher Demo": [0.356, 0.324, 0.0282, 0.835, 0.000122, 0.351, 0.331], "Everybody's Got Something To Hide Except Me And My Monkey - Esher Demo": [0.678, 0.587, 0.0398, 0.131, 0.0, 0.31, 0.704], 'Sexy Sadie - Esher Demo': [0.432, 0.283, 0.0528, 0.783, 1.95e-06, 0.358, 0.257], 'Revolution - Esher Demo': [0.723, 0.383, 0.0682, 0.294, 1.73e-05, 0.118, 0.569], 'Honey Pie - Esher Demo': [0.633, 0.47, 0.113, 0.4, 0.0, 0.261, 0.655], 'Cry Baby Cry - Esher Demo': [0.63, 0.277, 0.0529, 0.805, 0.0, 0.332, 0.717], 'Sour Milk Sea - Esher Demo': [0.662, 0.392, 0.0509, 0.86, 8.4e-06, 0.378, 0.402], 'Twist And Shout - Live / Remastered': [0.262, 0.777, 0.0382, 7.81e-05, 0.0, 0.462, 0.552], "She's A Woman - Live / Remastered": [0.36, 0.881, 0.0456, 0.0072, 7.75e-06, 0.872, 0.538], 'Dizzy Miss Lizzy - Live / Remastered': [0.419, 0.871, 0.0507, 0.0116, 5.91e-05, 0.584, 0.732], 'Ticket To Ride - Live / Remastered': [0.394, 0.806, 0.046, 0.00295, 0.0, 0.318, 0.346], "Can't Buy Me Love - Live / Remastered": [0.357, 0.854, 0.0489, 0.0089, 0.0, 0.9, 0.624], 'Things We Said Today - Live / Remastered': [0.306, 0.627, 0.0463, 0.0669, 0.0, 0.802, 0.488], 'Roll Over Beethoven - Live / Remastered': [0.199, 0.823, 0.0393, 0.00204, 0.0, 0.631, 0.73], 'Boys - Live / Remastered': [0.33, 0.941, 0.0586, 0.039, 0.000231, 0.853, 0.649], "A Hard Day's Night - Live / Remastered": [0.273, 0.875, 0.052, 0.0479, 8.5e-06, 0.917, 0.619], 'Help! - Live / Remastered': [0.328, 0.77, 0.0497, 0.024, 0.0, 0.834, 0.54], 'All My Loving - Live / Remastered': [0.205, 0.663, 0.0345, 0.0203, 0.0, 0.899, 0.725], 'She Loves You - Live / Remastered': [0.356, 0.859, 0.0869, 0.188, 0.0, 0.665, 0.479], 'Long Tall Sally - Live / Remastered': [0.38, 0.908, 0.048, 0.0283, 0.0, 0.578, 0.655], "You Can't Do That - Live / Bonus Track": [0.432, 0.706, 0.0333, 0.00224, 0.0, 0.834, 0.677], 'I Want To Hold Your Hand - Live / Bonus Track': [0.225, 0.833, 0.0592, 0.0175, 0.0, 0.948, 0.515], 'Everybody’s Trying To Be My Baby - Live / Bonus Track': [0.19, 0.896, 0.0517, 0.0548, 0.0, 0.488, 0.727], "Baby's In Black - Live / Bonus Track": [0.192, 0.756, 0.0543, 0.0183, 0.0, 0.918, 0.669], 'Get Back - Naked Version / Remastered 2013': [0.811, 0.629, 0.0429, 0.045, 0.00145, 0.0709, 0.54], 'Dig A Pony - Naked Version / Remastered 2013': [0.503, 0.502, 0.0414, 0.498, 0.0, 0.178, 0.499], 'For You Blue - Naked Version / Remastered 2013': [0.854, 0.625, 0.0946, 0.276, 0.00906, 0.13, 0.86], 'The Long And Winding Road - Naked Version / Remastered 2013': [0.569, 0.266, 0.0303, 0.945, 0.000523, 0.142, 0.279], 'Two Of Us - Naked Version / Remastered 2013': [0.699, 0.74, 0.042, 0.157, 9.98e-06, 0.147, 0.913], "I've Got A Feeling - Naked Version / Remastered 2013": [0.427, 0.73, 0.0756, 0.0311, 0.0, 0.135, 0.399], 'One After 909 - Naked Version / Remastered 2013': [0.408, 0.894, 0.123, 0.0376, 0.0, 0.319, 0.866], "Don't Let Me Down - Naked Version / Remastered 2013": [0.486, 0.489, 0.0365, 0.147, 8.63e-06, 0.135, 0.783], 'I Me Mine - Naked Version / Remastered 2013': [0.463, 0.604, 0.0412, 0.223, 0.0, 0.272, 0.758], 'Across The Universe - Naked Version / Remastered 2013': [0.359, 0.461, 0.0283, 0.509, 0.0, 0.224, 0.794], 'Let It Be - Naked Version / Remastered 2013': [0.47, 0.404, 0.036, 0.565, 1.68e-05, 0.152, 0.468], 'Yellow Submarine': [0.616, 0.636, 0.0325, 0.208, 0.0, 0.517, 0.654], 'Hey Bulldog': [0.568, 0.824, 0.0333, 0.0318, 0.0126, 0.197, 0.826], 'Eleanor Rigby': [0.521, 0.301, 0.0356, 0.836, 0.0, 0.398, 0.621], 'Love You To': [0.399, 0.649, 0.0508, 0.555, 4.53e-05, 0.113, 0.657], 'All Together Now': [0.606, 0.413, 0.0622, 0.116, 0.0, 0.352, 0.889], 'Lucy In The Sky With Diamonds': [0.269, 0.365, 0.0401, 0.138, 0.0, 0.0904, 0.357], 'Think For Yourself': [0.528, 0.591, 0.0282, 0.0162, 0.0, 0.0726, 0.796], "Sgt Pepper's Lonely Hearts Club Band": [0.438, 0.651, 0.0932, 0.218, 5.87e-05, 0.905, 0.624], 'With A Little Help From My Friends': [0.705, 0.427, 0.0353, 0.294, 0.0, 0.355, 0.827], "Baby You're A Rich Man": [0.663, 0.606, 0.0307, 0.0215, 0.0, 0.119, 0.412], 'Only A Northern Song': [0.364, 0.723, 0.0506, 0.000409, 0.000192, 0.0859, 0.78], 'All You Need Is Love': [0.364, 0.483, 0.0317, 0.281, 1.53e-06, 0.113, 0.7], "When I'm Sixty Four": [0.717, 0.224, 0.056, 0.683, 0.0, 0.13, 0.594], 'Nowhere Man': [0.534, 0.646, 0.0366, 0.0155, 1.42e-05, 0.142, 0.852], "It's All Too Much": [0.48, 0.81, 0.0382, 4.71e-05, 0.0327, 0.262, 0.208], 'And Here We Are Again - Live At The BBC For "Pop Go The Beatles" / 23rd July, 1963': [0.772, 0.306, 0.932, 0.773, 0.0, 0.281, 0.965], 'Words Of Love - Live At The BBC For "Pop Go The Beatles" / 20th August, 1963': [0.634, 0.607, 0.0277, 0.776, 0.000451, 0.176, 0.925], 'How About It, Gorgeous? - Live At The BBC For "Pop Go The Beatles" / 30th July, 1963': [0.649, 0.176, 0.942, 0.944, 0.0, 0.35, 0.923], 'Do You Want To Know A Secret - Live At The BBC For "Pop Go The Beatles" / 30th July, 1963': [0.55, 0.352, 0.0314, 0.65, 0.0, 0.471, 0.716], 'Lucille - Live At The BBC For "Pop Go The Beatles" / 17th September, 1963': [0.588, 0.843, 0.044, 0.752, 0.00291, 0.476, 0.91], 'Hey Paul…. - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.773, 0.16, 0.934, 0.852, 0.0, 0.153, 0.969], 'Anna (Go To Him) - Live At The BBC For "Pop Go The Beatles" / 27th August, 1963': [0.71, 0.473, 0.0331, 0.944, 0.0163, 0.257, 0.812], 'Hello! - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.736, 0.26, 0.943, 0.907, 0.0, 0.265, 0.973], 'Please Please Me - Live At The BBC For "Pop Go The Beatles" / 13th August, 1963': [0.526, 0.614, 0.0376, 0.796, 0.0, 0.213, 0.824], 'Misery - Live At The BBC For "Here We Go" / 12th March, 1963': [0.68, 0.535, 0.0346, 0.298, 0.0, 0.833, 0.857], 'I\'m Talking About You - Live At The BBC For "Saturday Club" / 16th March, 1963': [0.579, 0.776, 0.0419, 0.825, 0.753, 0.55, 0.95], 'A Real Treat - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.742, 0.254, 0.942, 0.845, 0.0, 0.171, 0.798], 'Boys - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.502, 0.609, 0.0365, 0.644, 0.0, 0.496, 0.792], 'Absolutely Fab - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.786, 0.289, 0.939, 0.905, 0.0, 0.202, 0.949], 'Chains - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.517, 0.576, 0.0434, 0.541, 0.0, 0.081, 0.924], 'Ask Me Why - Live At The BBC For "Pop Go The Beatles" / 24th September, 1963': [0.694, 0.266, 0.0342, 0.793, 9.12e-05, 0.0831, 0.874], 'Till There Was You - Live At The BBC For "Pop Go The Beatles" / 30th July, 1963': [0.703, 0.369, 0.0347, 0.859, 0.000101, 0.0844, 0.759], 'Lend Me Your Comb - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.592, 0.521, 0.0409, 0.529, 0.000441, 0.0875, 0.961], 'Lower 5E - Live At The BBC For "Pop Go The Beatles" / 10th September, 1963': [0.701, 0.149, 0.942, 0.962, 0.0, 0.151, 0.96], 'The Hippy Hippy Shake - Live At The BBC For "Pop Go The Beatles" / 10th September, 1963': [0.505, 0.763, 0.0763, 0.691, 0.00766, 0.173, 0.928], 'Roll Over Beethoven - Live At The BBC For "Pop Go The Beatles" / 3rd September, 1963': [0.387, 0.885, 0.0655, 0.961, 7.62e-05, 0.344, 0.892], 'There\'s A Place - Live At The BBC For "Pop Go The Beatles" / 3rd September, 1963': [0.514, 0.669, 0.0302, 0.633, 0.0, 0.623, 0.863], 'Bumper Bundle - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.715, 0.196, 0.943, 0.854, 0.0, 0.109, 0.906], 'P.S. I Love You - Live At The BBC For "Pop Go The Beatles" / 25th June, 1963': [0.551, 0.295, 0.0313, 0.451, 7.83e-06, 0.231, 0.736], 'Please Mister Postman - Live At The BBC For "Pop Go The Beatles" / 30th July 1963': [0.704, 0.549, 0.0346, 0.823, 0.0, 0.123, 0.922], 'Beautiful Dreamer - Live At The BBC For "Saturday Club" / 26th January, 1963': [0.437, 0.765, 0.0856, 0.84, 0.166, 0.2, 0.809], 'Devil In Her Heart - Live At The BBC For "Pop Go The Beatles" / 25th September, 1963': [0.713, 0.533, 0.0317, 0.788, 0.000262, 0.344, 0.901], 'The 49 Weeks - Live At The BBC For "Pop Go The Beatles" / 24th September, 1963': [0.861, 0.2, 0.92, 0.977, 0.0, 0.221, 0.98], 'Sure To Fall (In Love With You) - Live At The BBC For "Pop Go The Beatles" / 24th September, 1963': [0.656, 0.339, 0.03, 0.781, 0.00116, 0.177, 0.727], 'Never Mind, Eh? - Live At The BBC For "Pop Go The Beatles" / 24th September, 1963': [0.716, 0.164, 0.935, 0.961, 0.0, 0.356, 0.546], 'Twist And Shout - Live At The BBC For "Pop Go The Beatles" / 6th August, 1963': [0.649, 0.753, 0.0359, 0.599, 0.0, 0.0431, 0.932], 'Bye, Bye - Live At The BBC For "Pop Go The Beatles" / 24th September, 1963': [0.722, 0.201, 0.89, 0.979, 0.0, 0.154, 0.0], 'John - Pop Profile - Live At The BBC / 30th November, 1965': [0.594, 0.186, 0.873, 0.738, 0.0, 0.109, 0.819], 'George - Pop Profile - Live At The BBC / 30th November, 1965': [0.665, 0.118, 0.775, 0.786, 0.0, 0.192, 0.596], 'I Saw Her Standing There - Live At The BBC For "Saturday Club" / 5th October, 1963': [0.492, 0.506, 0.0348, 0.15, 3.51e-06, 0.118, 0.961], 'Glad All Over - Live At The BBC For "Saturday Club" / 24th August, 1963': [0.638, 0.581, 0.0429, 0.623, 0.0, 0.53, 0.959], 'Lift Lid Again - Live At The BBC For "Saturday Club" / 24th August, 1963': [0.678, 0.16, 0.947, 0.966, 0.0, 0.315, 0.864], 'I\'ll Get You - Live At The BBC For "Saturday Club" / 5th October, 1963': [0.753, 0.347, 0.0485, 0.307, 4.03e-06, 0.146, 0.951], 'She Loves You - Live At The BBC For "Saturday Club" / 5th October, 1963': [0.43, 0.522, 0.0288, 0.437, 0.0, 0.105, 0.905], 'Memphis, Tennessee - Live At The BBC For "Saturday Club" / 5th October, 1963': [0.764, 0.303, 0.0454, 0.228, 0.135, 0.109, 0.705], 'Happy Birthday Dear Saturday Club - Live At The BBC For "Saturday Club" / 5th October, 1963': [0.759, 0.486, 0.137, 0.722, 0.0, 0.105, 0.863], 'Now Hush, Hush - Live At The BBC For "Easy Beat" / 20th October, 1963': [0.469, 0.507, 0.587, 0.93, 0.0, 0.924, 0.432], 'From Me To You - Live At The BBC For "Easy Beat" / 20th October, 1963': [0.474, 0.785, 0.0502, 0.438, 9.61e-05, 0.533, 0.792], 'Money (That\'s What I Want) - Live At The BBC For "The Beatles Say From Us To You" / 26th December, 1963': [0.633, 0.671, 0.0377, 0.813, 0.00164, 0.174, 0.961], 'I Want To Hold Your Hand - Live At The BBC For "The Beatles Say From Us To You" / 26th December, 1963': [0.588, 0.744, 0.0356, 0.674, 5.99e-05, 0.336, 0.865], 'Brian Bathtubes - Live At The BBC For "Saturday Club" / 21st December, 1963': [0.715, 0.302, 0.942, 0.954, 0.0, 0.226, 0.795], 'This Boy - Live At The BBC For "Saturday Club" / 21st December, 1963': [0.633, 0.45, 0.0322, 0.809, 8.9e-05, 0.0922, 0.525], 'If I Wasn’t In America - Live At The BBC For "Saturday Club" / 15th February, 1964': [0.674, 0.54, 0.845, 0.915, 0.0012, 0.648, 0.958], 'I Got A Woman - Live At The BBC For "Saturday Club" / 4th April, 1964': [0.592, 0.563, 0.0505, 0.0257, 0.000995, 0.0751, 0.772], 'Long Tall Sally - Live At The BBC For "Top Gear" / 16th July, 1964': [0.407, 0.477, 0.0411, 0.0026, 0.00261, 0.0411, 0.782], 'Love Me Do - Mono / Remastered': [0.524, 0.88, 0.0987, 0.404, 0.000191, 0.154, 0.758], 'From Me To You - Mono / Remastered': [0.581, 0.821, 0.0318, 0.507, 0.0, 0.108, 0.968], 'She Loves You - Mono / Remastered': [0.377, 0.916, 0.0481, 0.26, 0.0, 0.074, 0.912], 'I Want To Hold Your Hand - Remastered 2015': [0.49, 0.715, 0.0476, 0.386, 0.0, 0.311, 0.866], "Can't Buy Me Love - Remastered 2015": [0.426, 0.76, 0.0414, 0.118, 0.0, 0.325, 0.955], "A Hard Day's Night - Remastered 2015": [0.546, 0.818, 0.031, 0.0487, 0.0, 0.0983, 0.756], 'I Feel Fine - Remastered 2015': [0.564, 0.827, 0.0283, 0.0904, 3.73e-06, 0.127, 0.912], 'Eight Days A Week - Remastered 2015': [0.636, 0.677, 0.0316, 0.265, 0.0, 0.215, 0.837], 'Ticket To Ride - Remastered 2015': [0.554, 0.904, 0.0536, 0.0638, 0.0, 0.259, 0.801], 'Help! - Remastered 2015': [0.602, 0.736, 0.0332, 0.149, 0.0, 0.0776, 0.819], 'Yesterday - Remastered 2015': [0.401, 0.189, 0.0287, 0.871, 0.0, 0.0968, 0.357], 'Day Tripper - Remastered 2015': [0.665, 0.782, 0.0307, 0.12, 4.19e-06, 0.125, 0.731], 'We Can Work It Out - Remastered 2015': [0.378, 0.677, 0.0298, 0.0635, 0.0, 0.183, 0.648], 'Paperback Writer - Remastered 2015': [0.472, 0.856, 0.0562, 0.147, 2.78e-06, 0.595, 0.841], 'Yellow Submarine - Remastered 2015': [0.66, 0.557, 0.03, 0.158, 0.0, 0.543, 0.788], 'Eleanor Rigby - Remastered 2015': [0.605, 0.314, 0.0327, 0.916, 0.0, 0.359, 0.836], 'Penny Lane - Remastered 2015': [0.679, 0.592, 0.0402, 0.0964, 0.0138, 0.16, 0.55], 'All You Need Is Love - Remastered 2015': [0.397, 0.625, 0.0347, 0.437, 5.47e-06, 0.263, 0.676], 'Hello, Goodbye - Remastered 2015': [0.498, 0.806, 0.0265, 0.402, 5.74e-06, 0.525, 0.664], 'Lady Madonna - Remastered 2015': [0.65, 0.777, 0.0294, 0.141, 0.343, 0.215, 0.574], 'Hey Jude - Remastered 2015': [0.386, 0.607, 0.0261, 0.0112, 1.38e-05, 0.088, 0.532], 'Get Back - Remastered 2015': [0.703, 0.636, 0.0376, 0.186, 0.0256, 0.0959, 0.218], 'The Ballad Of John And Yoko - Remastered 2015': [0.651, 0.892, 0.0313, 0.00465, 2.85e-06, 0.127, 0.962], 'Something - Remastered 2015': [0.449, 0.375, 0.0286, 0.302, 2.41e-06, 0.144, 0.466], 'Come Together - Remastered 2015': [0.559, 0.343, 0.0424, 0.0601, 0.138, 0.1, 0.183], 'Let It Be - Remastered 2015': [0.445, 0.436, 0.0332, 0.58, 2.63e-05, 0.112, 0.483], 'The Long And Winding Road - Remastered 2015': [0.326, 0.365, 0.0265, 0.667, 0.000523, 0.0718, 0.314], 'Beatles Greetings - Live At The BBC For "The Public Ear" / 3rd November, 1963': [0.0, 0.29, 0.0, 0.878, 0.0, 0.789, 0.0], 'From Us To You - Live At The BBC / Opening Theme From "From Us To You" / 1964': [0.643, 0.773, 0.0395, 0.0349, 0.0514, 0.08, 0.88], 'Riding On A Bus - Live At The BBC For "Top Gear" / 26th November, 1964': [0.646, 0.135, 0.904, 0.928, 0.0, 0.353, 0.624], 'I Got A Woman - Live At The BBC For "Pop Go The Beatles" / 13th August, 1963': [0.676, 0.661, 0.0348, 0.648, 0.0, 0.0966, 0.909], 'Too Much Monkey Business - Live At The BBC For "Pop Go The Beatles" / 10th September, 1963': [0.632, 0.766, 0.0601, 0.28, 1.87e-06, 0.086, 0.965], 'Keep Your Hands Off My Baby - Live At The BBC For "Saturday Club" / 26th January, 1963': [0.566, 0.629, 0.103, 0.0851, 0.833, 0.0795, 0.885], 'I\'ll Be On My Way - Live At The BBC For "Side By Side" / 24th June, 1963': [0.613, 0.485, 0.0306, 0.234, 2.32e-06, 0.142, 0.79], 'Young Blood - Live At The BBC For "Pop Go The Beatles" / 11th June, 1963': [0.474, 0.615, 0.0358, 0.701, 0.0, 0.926, 0.923], 'A Shot Of Rhythm And Blues - Live At The BBC For "Pop Go The Beatles" / 27th August, 1963': [0.699, 0.752, 0.0354, 0.879, 0.000236, 0.265, 0.968], 'Sure To Fall (In Love With You) - Live At The BBC For "Pop Go The Beatles" / 18th June, 1963': [0.491, 0.53, 0.03, 0.351, 7.73e-06, 0.588, 0.897], 'Some Other Guy - Live At The BBC For "Easy Beat" / 23rd June, 1963': [0.451, 0.809, 0.0425, 0.00388, 0.442, 0.139, 0.894], 'Thank You Girl - Live At The BBC For "Easy Beat" / 23rd June, 1963': [0.405, 0.722, 0.0317, 0.0183, 0.0, 0.122, 0.932], 'Sha La La La La! - Live At The BBC For "Pop Go The Beatles" / 11th June, 1963': [0.803, 0.349, 0.887, 0.583, 0.0, 0.457, 0.95], 'Baby It\'s You - Live At The BBC For "Pop Go The Beatles" / 11th June, 1963': [0.531, 0.525, 0.0298, 0.326, 0.0, 0.432, 0.786], 'That\'s All Right (Mama) - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.671, 0.657, 0.0299, 0.609, 0.596, 0.118, 0.957], 'Carol - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.484, 0.706, 0.0352, 0.311, 6.55e-06, 0.0838, 0.979], 'What Is It, George? - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.744, 0.173, 0.93, 0.784, 0.0, 0.506, 0.649], 'Soldier Of Love - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.742, 0.413, 0.0401, 0.319, 2.62e-06, 0.305, 0.969], 'A Little Rhyme - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.744, 0.182, 0.954, 0.82, 0.0, 0.276, 0.88], 'Clarabella - Live At The BBC For "Pop Go The Beatles" / 16th July, 1963': [0.521, 0.63, 0.0728, 0.61, 0.0, 0.597, 0.883], 'I\'m Gonna Sit Right Down And Cry (Over You) - Live At The BBC For "Pop Go The Beatles" / 6th August, 1963': [0.638, 0.651, 0.047, 0.357, 9.36e-06, 0.0737, 0.942], 'Crying, Waiting, Hoping - Live At The BBC For "Pop Go The Beatles" / 6th August, 1963': [0.646, 0.312, 0.0358, 0.689, 0.0, 0.1, 0.895], 'Dear Wack! - Live At The BBC For "Saturday Club" / 24th August, 1963': [0.626, 0.15, 0.836, 0.928, 0.0, 0.196, 0.934], 'You Really Got A Hold On Me - Live At The BBC For "Saturday Club" / 24th August, 1963': [0.649, 0.294, 0.0321, 0.728, 0.0, 0.0716, 0.738], 'To Know Her Is To Love Her - Live At The BBC For "Pop Go The Beatles" / 6th August, 1963': [0.45, 0.34, 0.0337, 0.777, 0.0, 0.22, 0.456], 'A Taste Of Honey - Live At The BBC For "Pop Go The Beatles" / 23rd July, 1963': [0.506, 0.285, 0.0313, 0.737, 0.0, 0.333, 0.607], 'Long Tall Sally - Live At The BBC For "Pop Go The Beatles" / 13th August, 1963': [0.593, 0.666, 0.0485, 0.267, 0.000226, 0.126, 0.877], 'I Saw Her Standing There - Live At The BBC For "Easy Beat" / 20th October 1963': [0.459, 0.732, 0.0389, 0.233, 0.0, 0.943, 0.939], 'The Honeymoon Song - Live At The BBC For "Pop Go The Beatles" / 6th August, 1963': [0.727, 0.323, 0.0498, 0.724, 0.0, 0.0794, 0.8], 'Johnny B Goode - Live At The BBC For "Saturday Club" / 15th February, 1964': [0.492, 0.737, 0.0442, 0.0281, 1.43e-05, 0.239, 0.961], 'Memphis, Tennessee - Live At The BBC For "Pop Go The Beatles" / 30th July, 1963': [0.771, 0.41, 0.045, 0.611, 0.00265, 0.0875, 0.539], 'Lucille - Live At The BBC For "Saturday Club" / 5th October, 1963': [0.632, 0.651, 0.0306, 0.397, 0.0, 0.293, 0.97], 'Can\'t Buy Me Love - Live At The BBC For "From Us To You Say The Beatles" / 10th March, 1964': [0.53, 0.597, 0.0342, 0.316, 0.0, 0.451, 0.946], 'From Fluff To You - Live At The BBC For "From Us To You Say The Beatles" / 10th March, 1964': [0.719, 0.0597, 0.902, 0.753, 6.58e-06, 0.12, 0.72], 'Till There Was You - Live At The BBC For "From Us To You Say The Beatles" / 10th March, 1964': [0.571, 0.329, 0.0363, 0.618, 1.79e-06, 0.0876, 0.571], 'Crinsk Dee Night - Live At The BBC For "Top Gear" / 16th July, 1964': [0.678, 0.11, 0.456, 0.807, 0.0, 0.303, 0.699], 'A Hard Day\'s Night - Live At The BBC For "Top Gear" / 16th July, 1964': [0.551, 0.525, 0.0363, 0.0179, 0.0, 0.147, 0.796], 'Ringo? Yep! - Live At The BBC For "From Us To You Say The Beatles" / 30th March, 1964': [0.0, 0.0613, 0.0, 0.318, 0.0, 0.317, 0.0], 'I Wanna Be Your Man - Live At The BBC For "From Us To You Say The Beatles" / 30th March, 1964': [0.435, 0.82, 0.0804, 0.426, 8.23e-06, 0.124, 0.701], 'Just A Rumour - Live At The BBC For "From Us To You Say The Beatles" / 30th March, 1964': [0.737, 0.0585, 0.918, 0.589, 0.0, 0.392, 0.923], 'Roll Over Beethoven - Live At The BBC For "From Us To You Say The Beatles" / 30th March, 1964': [0.468, 0.644, 0.044, 0.27, 0.000156, 0.185, 0.966], 'All My Loving - Live At The BBC For "From Us To You Say The Beatles" / 30th March, 1964': [0.449, 0.49, 0.0304, 0.117, 0.00462, 0.316, 0.879], 'Things We Said Today - Live At The BBC For "Top Gear" / 16th July, 1964': [0.649, 0.49, 0.034, 0.265, 0.0, 0.0662, 0.779], 'She\'s A Woman - Live At The BBC For "Top Gear" / 26th November, 1964': [0.759, 0.378, 0.0402, 0.138, 0.00016, 0.0671, 0.907], 'Sweet Little Sixteen - Live At The BBC For "Pop Go The Beatles" / 23rd July, 1963': [0.493, 0.637, 0.0455, 0.403, 0.0, 0.071, 0.976], '1822! - Live At The BBC For "Pop Go The Beatles" / 23rd July, 1963': [0.0, 0.0923, 0.0, 0.953, 0.0, 0.155, 0.0], 'Lonesome Tears In My Eyes - Live At The BBC For "Pop Go The Beatles" / 23rd July, 1963': [0.579, 0.481, 0.0398, 0.46, 0.0459, 0.0742, 0.963], 'Nothin\' Shakin\' - Live At The BBC For "Pop Go The Beatles" / 23rd July, 1963': [0.761, 0.614, 0.0374, 0.518, 0.0, 0.179, 0.979], 'The Hippy Hippy Shake - Live At The BBC For "Pop Go The Beatles" / 30th July, 1963': [0.541, 0.6, 0.0311, 0.504, 0.0, 0.238, 0.958], 'Glad All Over - Live At The BBC For "Pop Go The Beatles" / 20th August, 1963': [0.603, 0.616, 0.0346, 0.583, 0.0, 0.454, 0.913], 'Two Of Us - Remastered 2009': [0.726, 0.702, 0.032, 0.0974, 3.46e-05, 0.239, 0.931], 'Dig A Pony - Remastered 2009': [0.538, 0.478, 0.0388, 0.472, 0.0, 0.17, 0.527], 'Across The Universe - Remastered 2009': [0.257, 0.412, 0.0287, 0.361, 8.68e-05, 0.0702, 0.858], 'I Me Mine - Remastered 2009': [0.291, 0.638, 0.0554, 0.179, 0.0, 0.101, 0.525], 'Dig It - Remastered 2009': [0.54, 0.489, 0.117, 0.659, 0.0019, 0.457, 0.67], 'Let It Be - Remastered 2009': [0.443, 0.403, 0.0322, 0.631, 0.0, 0.111, 0.41], 'Maggie Mae - Remastered 2009': [0.516, 0.515, 0.215, 0.379, 0.0, 0.103, 0.524], "I've Got A Feeling - Remastered 2009": [0.44, 0.609, 0.0358, 0.0715, 0.0, 0.582, 0.364], 'One After 909 - Remastered 2009': [0.554, 0.828, 0.0739, 0.0307, 0.0, 0.907, 0.888], 'The Long And Winding Road - Remastered 2009': [0.299, 0.329, 0.0279, 0.756, 0.0105, 0.0559, 0.392], 'For You Blue - Remastered 2009': [0.88, 0.556, 0.0855, 0.24, 0.0483, 0.24, 0.955], 'Get Back - Remastered 2009': [0.761, 0.592, 0.0586, 0.492, 0.00626, 0.61, 0.332], 'Come Together - Remastered 2009': [0.533, 0.376, 0.0393, 0.0302, 0.248, 0.0926, 0.187], 'Something - Remastered 2009': [0.396, 0.338, 0.0304, 0.198, 2.62e-06, 0.138, 0.377], "Maxwell's Silver Hammer - Remastered 2009": [0.764, 0.426, 0.0405, 0.629, 0.000119, 0.132, 0.726], 'Oh! Darling - Remastered 2009': [0.422, 0.656, 0.0397, 0.0126, 0.0279, 0.268, 0.549], "Octopus's Garden - Remastered 2009": [0.626, 0.512, 0.0247, 0.155, 4.65e-05, 0.215, 0.73], "I Want You (She's So Heavy) - Remastered 2009": [0.37, 0.515, 0.0321, 0.0233, 0.205, 0.0966, 0.42], 'Here Comes The Sun - Remastered 2009': [0.557, 0.54, 0.0347, 0.0339, 0.00248, 0.179, 0.394], 'Because - Remastered 2009': [0.328, 0.249, 0.0272, 0.754, 5.77e-06, 0.134, 0.162], 'You Never Give Me Your Money - Remastered 2009': [0.335, 0.416, 0.0348, 0.345, 0.136, 0.116, 0.223], 'Sun King - Remastered 2009': [0.483, 0.177, 0.0277, 0.284, 0.499, 0.116, 0.192], 'Mean Mr Mustard - Remastered 2009': [0.676, 0.596, 0.0272, 0.0699, 1.96e-05, 0.351, 0.97], 'Polythene Pam - Remastered 2009': [0.474, 0.939, 0.199, 0.00472, 0.674, 0.19, 0.499], 'She Came In Through The Bathroom Window - Remastered 2009': [0.53, 0.72, 0.0338, 0.00376, 3.38e-05, 0.314, 0.69], 'Golden Slumbers - Remastered 2009': [0.419, 0.152, 0.0277, 0.326, 0.0034, 0.113, 0.239], 'Carry That Weight - Remastered 2009': [0.543, 0.458, 0.0274, 0.226, 0.00663, 0.298, 0.704], 'The End - Remastered 2009': [0.647, 0.487, 0.0689, 0.00101, 0.285, 0.138, 0.222], 'Her Majesty - Remastered 2009': [0.703, 0.215, 0.0892, 0.728, 1.23e-06, 0.13, 0.522], 'Yellow Submarine - Remastered 2009': [0.604, 0.549, 0.0389, 0.531, 0.0, 0.438, 0.696], 'Only A Northern Song - Remastered 2009': [0.175, 0.731, 0.0705, 0.000341, 0.000751, 0.0846, 0.801], 'All Together Now - Remastered 2009': [0.587, 0.431, 0.0686, 0.0722, 0.0, 0.561, 0.778], 'Hey Bulldog - Remastered 2009': [0.587, 0.617, 0.028, 0.0197, 0.639, 0.199, 0.947], "It's All Too Much - Remastered 2009": [0.39, 0.781, 0.0364, 4.3e-05, 0.412, 0.1, 0.199], 'All You Need Is Love - Remastered 2009': [0.4, 0.48, 0.0295, 0.346, 3.1e-05, 0.155, 0.653], 'Pepperland - Remastered 2009': [0.354, 0.281, 0.032, 0.661, 0.763, 0.243, 0.367], 'Sea Of Time - Remastered 2009': [0.237, 0.096, 0.0387, 0.971, 0.0961, 0.203, 0.0629], 'Sea Of Holes - Remastered 2009': [0.249, 0.0994, 0.04, 0.959, 0.926, 0.454, 0.0357], 'Sea Of Monsters - Remastered 2009': [0.253, 0.166, 0.0346, 0.945, 0.862, 0.135, 0.0489], 'March Of The Meanies - Remastered 2009': [0.538, 0.378, 0.0387, 0.934, 0.758, 0.456, 0.16], 'Pepperland Laid Waste - Remastered 2009': [0.269, 0.0902, 0.042, 0.877, 0.821, 0.302, 0.0462], 'Yellow Submarine In Pepperland - Remastered 2009': [0.406, 0.219, 0.0359, 0.808, 0.599, 0.15, 0.271], 'Back In The U.S.S.R. - Remastered 2009': [0.48, 0.969, 0.0854, 0.069, 0.00022, 0.0608, 0.494], 'Dear Prudence - Remastered 2009': [0.388, 0.529, 0.0436, 0.16, 0.0681, 0.108, 0.332], 'Glass Onion - Remastered 2009': [0.523, 0.51, 0.0514, 0.0123, 8.38e-05, 0.108, 0.789], 'Ob-La-Di, Ob-La-Da - Remastered 2009': [0.818, 0.728, 0.0314, 0.232, 0.0642, 0.251, 0.975], 'Wild Honey Pie - Remastered 2009': [0.792, 0.763, 0.0506, 0.425, 0.627, 0.789, 0.152], 'The Continuing Story Of Bungalow Bill - Remastered 2009': [0.621, 0.459, 0.104, 0.224, 0.0, 0.656, 0.648], 'While My Guitar Gently Weeps - Remastered 2009': [0.453, 0.654, 0.0317, 0.0205, 0.00395, 0.172, 0.702], 'Happiness Is A Warm Gun - Remastered 2009': [0.431, 0.54, 0.0589, 0.523, 1.34e-06, 0.169, 0.44], 'Martha My Dear - Remastered 2009': [0.472, 0.292, 0.0321, 0.872, 0.0007, 0.128, 0.824], "I'm So Tired - Remastered 2009": [0.433, 0.483, 0.175, 0.656, 2.64e-05, 0.539, 0.717], 'Blackbird - Remastered 2009': [0.686, 0.127, 0.0342, 0.754, 1.41e-05, 0.0573, 0.372], 'Piggies - Remastered 2009': [0.542, 0.445, 0.0303, 0.643, 0.0, 0.516, 0.52], 'Rocky Raccoon - Remastered 2009': [0.599, 0.363, 0.0315, 0.333, 0.0, 0.243, 0.648], "Don't Pass Me By - Remastered 2009": [0.599, 0.537, 0.0254, 0.594, 1.92e-05, 0.194, 0.946], "Why Don't We Do It In The Road? - Remastered 2009": [0.589, 0.522, 0.0521, 0.53, 0.000243, 0.0993, 0.957], 'I Will - Remastered 2009': [0.631, 0.387, 0.0262, 0.742, 0.000406, 0.0822, 0.832], 'Julia - Remastered 2009': [0.512, 0.0953, 0.0304, 0.828, 0.000268, 0.117, 0.193], 'Birthday - Remastered 2009': [0.541, 0.727, 0.0384, 7.37e-05, 0.717, 0.232, 0.91], 'Yer Blues - Remastered 2009': [0.327, 0.525, 0.0475, 0.000875, 0.259, 0.133, 0.435], "Mother Nature's Son - Remastered 2009": [0.608, 0.178, 0.0264, 0.634, 0.352, 0.166, 0.401], "Everybody's Got Something To Hide Except Me And My Monkey - Remastered 2009": [0.523, 0.731, 0.0559, 0.0207, 0.000464, 0.081, 0.81], 'Sexy Sadie - Remastered 2009': [0.577, 0.624, 0.0285, 0.33, 0.00622, 0.117, 0.671], 'Helter Skelter - Remastered 2009': [0.166, 0.831, 0.0894, 0.000606, 0.846, 0.811, 0.281], 'Long, Long, Long - Remastered 2009': [0.349, 0.129, 0.0294, 0.539, 0.405, 0.192, 0.163], 'Revolution 1 - Remastered 2009': [0.386, 0.626, 0.0377, 0.0785, 0.00164, 0.621, 0.695], 'Honey Pie - Remastered 2009': [0.537, 0.297, 0.0822, 0.88, 1.5e-05, 0.145, 0.553], 'Savoy Truffle - Remastered 2009': [0.623, 0.544, 0.0536, 0.00814, 0.309, 0.134, 0.793], 'Cry Baby Cry - Remastered 2009': [0.517, 0.529, 0.0323, 0.115, 0.00396, 0.174, 0.362], 'Revolution 9 - Remastered 2009': [0.208, 0.615, 0.342, 0.769, 0.118, 0.824, 0.101], 'Good Night - Remastered 2009': [0.146, 0.355, 0.0352, 0.865, 0.835, 0.114, 0.178], 'Magical Mystery Tour - Remastered 2009': [0.297, 0.613, 0.0931, 0.0385, 3.98e-05, 0.0734, 0.338], 'The Fool On The Hill - Remastered 2009': [0.356, 0.26, 0.0314, 0.913, 2.1e-06, 0.105, 0.49], 'Flying - Remastered 2009': [0.551, 0.395, 0.0501, 0.368, 0.88, 0.0932, 0.261], 'Blue Jay Way - Remastered 2009': [0.294, 0.407, 0.0308, 0.0258, 0.000219, 0.0852, 0.0378], 'Your Mother Should Know - Remastered 2009': [0.698, 0.293, 0.0332, 0.784, 0.0106, 0.0936, 0.724], 'I Am The Walrus - Remastered 2009': [0.401, 0.556, 0.0513, 0.0828, 0.000359, 0.589, 0.654], 'Hello, Goodbye - Remastered 2009': [0.479, 0.733, 0.0256, 0.306, 2.4e-05, 0.414, 0.806], 'Strawberry Fields Forever - Remastered 2009': [0.39, 0.502, 0.178, 0.336, 0.000138, 0.0713, 0.289], 'Penny Lane - Remastered 2009': [0.651, 0.488, 0.0316, 0.212, 0.026, 0.136, 0.49], "Baby, You're A Rich Man - Remastered 2009": [0.619, 0.625, 0.036, 0.0119, 1.05e-06, 0.152, 0.467], "Sgt. Pepper's Lonely Hearts Club Band - Remix": [0.36, 0.858, 0.106, 0.0308, 0.0115, 0.845, 0.522], 'With A Little Help From My Friends - Remix': [0.673, 0.639, 0.0274, 0.122, 0.0, 0.22, 0.75], 'Lucy In The Sky With Diamonds - Remix': [0.331, 0.521, 0.0379, 0.101, 0.0, 0.154, 0.432], 'Getting Better - Remix': [0.591, 0.551, 0.0308, 0.07, 0.0, 0.0942, 0.81], 'Fixing A Hole - Remix': [0.415, 0.665, 0.0448, 0.483, 0.0, 0.0663, 0.523], "She's Leaving Home - Remix": [0.376, 0.319, 0.031, 0.881, 0.0, 0.124, 0.217], 'Being For The Benefit Of Mr. Kite! - Remix': [0.247, 0.752, 0.0559, 0.103, 0.0, 0.107, 0.593], 'Within You Without You - Remix': [0.291, 0.551, 0.0345, 0.357, 6.16e-05, 0.239, 0.373], "When I'm Sixty-Four - Remix": [0.696, 0.308, 0.0557, 0.508, 0.0, 0.142, 0.564], 'Lovely Rita - Remix': [0.578, 0.687, 0.0241, 0.00726, 0.0, 0.0936, 0.659], 'Good Morning Good Morning - Remix': [0.505, 0.797, 0.0897, 0.000302, 0.0, 0.484, 0.825], "Sgt. Pepper's Lonely Hearts Club Band (Reprise) - Remix": [0.576, 0.951, 0.0521, 0.000249, 0.214, 0.446, 0.789], 'A Day In The Life - Remix': [0.377, 0.453, 0.0462, 0.268, 0.000141, 0.901, 0.121], "Sgt. Pepper's Lonely Hearts Club Band - Take 9 And Speech": [0.597, 0.66, 0.0771, 0.352, 0.00913, 0.693, 0.556], 'With A Little Help From My Friends - Take 1 / False Start And Take 2 / Instrumental': [0.915, 0.51, 0.121, 0.254, 0.479, 0.138, 0.356], 'Lucy In The Sky With Diamonds - Take 1': [0.423, 0.361, 0.0481, 0.749, 4.31e-05, 0.106, 0.328], 'Getting Better - Take 1 / Instrumental And Speech At The End': [0.718, 0.487, 0.0839, 0.412, 0.048, 0.306, 0.382], 'Fixing A Hole - Speech And Take 3': [0.622, 0.454, 0.0573, 0.185, 7.47e-05, 0.038, 0.613], "She's Leaving Home - Take 1 / Instrumental": [0.442, 0.0782, 0.0374, 0.931, 0.895, 0.108, 0.198], 'Being For The Benefit Of Mr. Kite! - Take 4': [0.595, 0.378, 0.0488, 0.482, 8.96e-06, 0.432, 0.536], 'Within You Without You - Take 1 / Indian Instruments': [0.383, 0.432, 0.0409, 0.699, 0.177, 0.133, 0.184], "When I'm Sixty-Four - Take 2": [0.816, 0.218, 0.159, 0.32, 0.0, 0.0846, 0.434], 'Lovely Rita - Speech And Take 9': [0.671, 0.645, 0.04, 0.384, 0.0, 0.382, 0.653], 'Good Morning Good Morning - Take 8': [0.749, 0.435, 0.0977, 0.0231, 0.000195, 0.135, 0.79], "Sgt. Pepper's Lonely Hearts Club Band (Reprise) - Speech And Take 8": [0.787, 0.762, 0.0818, 0.268, 8.09e-05, 0.0707, 0.679], 'A Day In The Life - Take 1 With Hums': [0.483, 0.326, 0.0379, 0.919, 0.000172, 0.254, 0.192], 'Strawberry Fields Forever - Take 7': [0.369, 0.402, 0.243, 0.754, 9.96e-05, 0.115, 0.346], 'Strawberry Fields Forever - Take 26': [0.448, 0.855, 0.184, 0.219, 0.000455, 0.34, 0.342], 'Strawberry Fields Forever - Stereo Mix 2015': [0.413, 0.722, 0.273, 0.385, 0.000335, 0.0994, 0.228], 'Penny Lane - Take 6 / Instrumental': [0.901, 0.548, 0.0996, 0.667, 0.837, 0.161, 0.135], 'Penny Lane - Stereo Mix 2017': [0.617, 0.577, 0.0388, 0.0644, 0.00157, 0.106, 0.627], "Sgt. Pepper's Lonely Hearts Club Band - Remastered 2009": [0.451, 0.673, 0.0734, 0.0857, 0.00225, 0.797, 0.583], 'With A Little Help From My Friends - Remastered 2009': [0.705, 0.515, 0.0337, 0.365, 0.0, 0.389, 0.803], 'Lucy In The Sky With Diamonds - Remastered 2009': [0.311, 0.325, 0.0283, 0.0469, 0.0, 0.139, 0.668], 'Getting Better - Remastered 2009': [0.632, 0.471, 0.0348, 0.313, 5.08e-06, 0.0712, 0.764], 'Fixing A Hole - Remastered 2009': [0.469, 0.507, 0.0451, 0.382, 0.0, 0.0783, 0.619], "She's Leaving Home - Remastered 2009": [0.381, 0.214, 0.0337, 0.944, 0.0, 0.106, 0.393], 'Being For The Benefit Of Mr. Kite! - Remastered 2009': [0.526, 0.413, 0.0446, 0.338, 0.0, 0.0992, 0.71], 'Within You Without You - Remastered 2009': [0.352, 0.479, 0.0338, 0.339, 0.000353, 0.486, 0.354], "When I'm Sixty Four - Remastered 2009": [0.704, 0.241, 0.0476, 0.625, 2.8e-05, 0.0868, 0.661], 'Lovely Rita - Remastered 2009': [0.61, 0.471, 0.0261, 0.0634, 0.0, 0.118, 0.63], 'Good Morning Good Morning - Remastered 2009': [0.542, 0.618, 0.139, 0.0858, 0.0, 0.772, 0.93], "Sgt. Pepper's Lonely Hearts Club Band - Reprise / Remastered 2009": [0.653, 0.889, 0.0453, 0.0061, 0.589, 0.892, 0.879], 'A Day In The Life - Remastered 2009': [0.364, 0.457, 0.0675, 0.29, 0.000106, 0.922, 0.175], 'Strawberry Fields Forever - Take 1': [0.623, 0.132, 0.0377, 0.829, 8.74e-05, 0.193, 0.181], 'Strawberry Fields Forever - Take 4': [0.527, 0.339, 0.054, 0.696, 4.48e-06, 0.139, 0.314], 'Penny Lane - Vocal Overdubs And Speech': [0.879, 0.143, 0.261, 0.7, 0.00184, 0.452, 0.393], 'A Day In The Life - Take 1': [0.508, 0.311, 0.0403, 0.922, 8.74e-05, 0.125, 0.2], 'A Day In The Life - Take 2': [0.5, 0.205, 0.0437, 0.905, 6.72e-05, 0.345, 0.171], 'A Day In The Life - Orchestra Overdub': [0.243, 0.57, 0.042, 0.819, 0.659, 0.297, 0.184], 'A Day In The Life - Hummed Last Chord / Takes 8, 9, 10 And 11': [0.423, 0.226, 0.516, 0.869, 0.0, 0.358, 0.325], 'A Day In The Life - The Last Chord': [0.438, 0.0124, 0.0516, 0.911, 6.71e-05, 0.11, 0.137], "Sgt. Pepper's Lonely Hearts Club Band - Take 1 / Instrumental": [0.695, 0.747, 0.0547, 0.717, 0.465, 0.0963, 0.141], 'Good Morning Good Morning - Take 1 / Instrumental Breakdown': [0.762, 0.582, 0.0847, 0.0275, 0.301, 0.176, 0.0765], 'Fixing A Hole - Take 1': [0.572, 0.409, 0.048, 0.177, 2.87e-05, 0.0721, 0.632], 'Being For The Benefit Of Mr. Kite! - Speech From Before Take 1 / Take 4 And Speech At End': [0.602, 0.302, 0.0606, 0.495, 6.1e-06, 0.233, 0.527], 'Being For The Benefit Of Mr. Kite! - Take 7': [0.6, 0.326, 0.101, 0.606, 0.0, 0.185, 0.574], 'Lucy In The Sky With Diamonds - Take 1 And Speech At The End': [0.46, 0.297, 0.0485, 0.724, 3.96e-05, 0.0799, 0.337], 'Lucy In The Sky With Diamonds - Speech, False Start And Take 5': [0.475, 0.384, 0.119, 0.723, 6.88e-06, 0.158, 0.401], 'Getting Better - Take 12': [0.769, 0.488, 0.0438, 0.124, 0.00063, 0.0715, 0.705], 'Within You Without You - George Coaching The Musicians': [0.36, 0.313, 0.0401, 0.828, 2.61e-05, 0.155, 0.315], "She's Leaving Home - Take 6 / Instrumental": [0.47, 0.0693, 0.038, 0.928, 0.681, 0.146, 0.285], 'Getting Better': [0.59, 0.444, 0.0394, 0.131, 5.06e-05, 0.13, 0.718], 'Taxman - Remastered 2009': [0.469, 0.596, 0.0749, 0.0101, 0.0, 0.321, 0.783], 'Eleanor Rigby - Remastered 2009': [0.581, 0.28, 0.0381, 0.936, 1.56e-05, 0.305, 0.813], "I'm Only Sleeping - Remastered 2009": [0.494, 0.47, 0.0301, 0.21, 0.0, 0.28, 0.698], 'Love You To - Remastered 2009': [0.391, 0.593, 0.0413, 0.7, 0.000249, 0.0829, 0.707], 'Here, There And Everywhere - Remastered 2009': [0.29, 0.247, 0.0383, 0.921, 0.00565, 0.0917, 0.36], 'She Said She Said - Remastered 2009': [0.329, 0.678, 0.0344, 0.103, 0.168, 0.843, 0.672], 'Good Day Sunshine - Remastered 2009': [0.77, 0.496, 0.0468, 0.719, 0.0, 0.123, 0.571], 'And Your Bird Can Sing - Remastered 2009': [0.555, 0.65, 0.0282, 0.0067, 0.0491, 0.236, 0.966], 'For No One - Remastered 2009': [0.483, 0.351, 0.0278, 0.783, 0.0, 0.115, 0.713], 'Doctor Robert - Remastered 2009': [0.394, 0.64, 0.0488, 0.164, 0.139, 0.153, 0.574], 'I Want To Tell You - Remastered 2009': [0.493, 0.525, 0.0315, 0.182, 4.4e-06, 0.148, 0.449], 'Got To Get You Into My Life - Remastered 2009': [0.521, 0.524, 0.0302, 0.42, 9.27e-06, 0.0838, 0.711], 'Tomorrow Never Knows - Remastered 2009': [0.38, 0.829, 0.0405, 8.4e-05, 0.00208, 0.342, 0.0534], 'Drive My Car - Remastered 2009': [0.754, 0.571, 0.0421, 0.0173, 0.0, 0.3, 0.82], 'Norwegian Wood (This Bird Has Flown) - Remastered 2009': [0.425, 0.369, 0.0379, 0.183, 0.00126, 0.112, 0.65], "You Won't See Me - Remastered 2009": [0.488, 0.539, 0.0304, 0.136, 0.0, 0.0538, 0.473], 'Nowhere Man - Remastered 2009': [0.534, 0.624, 0.0462, 0.00797, 0.0, 0.128, 0.687], 'Think For Yourself - Remastered 2009': [0.529, 0.567, 0.0282, 0.0124, 0.0, 0.0729, 0.799], 'The Word - Remastered 2009': [0.59, 0.641, 0.0416, 0.00682, 0.0, 0.0553, 0.792], 'Michelle - Remastered 2009': [0.578, 0.29, 0.0341, 0.652, 0.0, 0.105, 0.666], 'What Goes On - Remastered 2009': [0.67, 0.776, 0.0749, 0.0168, 3.01e-06, 0.0732, 0.765], 'Girl - Remastered 2009': [0.699, 0.294, 0.0294, 0.587, 0.0, 0.117, 0.893], "I'm Looking Through You - Remastered 2009": [0.667, 0.498, 0.0364, 0.00772, 0.0, 0.0481, 0.774], 'In My Life - Remastered 2009': [0.688, 0.435, 0.0323, 0.449, 0.0, 0.113, 0.435], 'Wait - Remastered 2009': [0.552, 0.487, 0.04, 0.0175, 0.0, 0.105, 0.657], 'If I Needed Someone - Remastered 2009': [0.492, 0.755, 0.0331, 0.00748, 0.0, 0.102, 0.796], 'Run For Your Life - Remastered 2009': [0.563, 0.705, 0.103, 0.0103, 0.0, 0.0552, 0.822], 'Help! - Remastered 2009': [0.534, 0.725, 0.0332, 0.188, 0.0, 0.0994, 0.763], 'The Night Before - Remastered 2009': [0.461, 0.762, 0.0379, 0.0537, 0.0, 0.238, 0.632], "You've Got To Hide Your Love Away - Remastered 2009": [0.398, 0.392, 0.0282, 0.207, 0.0, 0.102, 0.378], 'I Need You - Remastered 2009': [0.601, 0.489, 0.0455, 0.277, 0.0, 0.0988, 0.638], 'Another Girl - Remastered 2009': [0.516, 0.638, 0.0314, 0.238, 0.0, 0.333, 0.822], "You're Going To Lose That Girl - Remastered 2009": [0.533, 0.66, 0.0337, 0.346, 0.0, 0.26, 0.818], 'Ticket To Ride - Remastered 2009': [0.519, 0.85, 0.0678, 0.0457, 0.0, 0.233, 0.749], 'Act Naturally - Remastered 2009': [0.702, 0.447, 0.0308, 0.366, 0.0, 0.153, 0.944], "It's Only Love - Remastered 2009": [0.629, 0.551, 0.0256, 0.22, 0.0, 0.0432, 0.685], 'You Like Me Too Much - Remastered 2009': [0.555, 0.57, 0.0302, 0.415, 0.0, 0.191, 0.899], 'Tell Me What You See - Remastered 2009': [0.592, 0.796, 0.0429, 0.061, 0.0, 0.131, 0.7], "I've Just Seen A Face - Remastered 2009": [0.647, 0.56, 0.0387, 0.723, 0.0, 0.118, 0.913], 'Yesterday - Remastered 2009': [0.332, 0.179, 0.0326, 0.879, 0.0, 0.0886, 0.315], 'Dizzy Miss Lizzy - Remastered 2009': [0.373, 0.934, 0.0591, 0.116, 0.00445, 0.0962, 0.918]}

In [21]:
beatles_features = beatles_dic.values()
justin_features = justin_dic.values()

j_song = [e for e in justin_dic.keys()]
b_song = [e for e in beatles_dic.keys()]

In [18]:
jf = [e for e in justin_features]
bf = [e for e in beatles_features]

In [25]:
bet_pre = model.predict(np.asarray(bf))
for e in bet_pre:
    if e == 1:
        print(bet_pre.tolist().index(e))

29
29
29
29
29
29
29
29
29
29
29
29
29
29


In [27]:
b_song[29]

'Come Together - Take 5'

In [33]:
j_pre = model.predict(np.asarray(jf))
print(j_pre)

[[3.57914723e-06]
 [9.69815552e-02]
 [9.93476450e-01]
 [5.87160029e-02]
 [9.99842525e-01]
 [1.61768440e-02]
 [1.43485302e-02]
 [7.68865868e-02]
 [9.99930143e-01]
 [9.99971032e-01]
 [1.15389854e-01]
 [5.01806801e-03]
 [1.75203476e-02]
 [1.02301216e-04]
 [1.33304149e-02]
 [2.74274107e-02]
 [6.21370137e-01]
 [4.28088080e-15]
 [1.00000000e+00]
 [9.69593644e-01]
 [9.99740303e-01]
 [9.98576760e-01]
 [3.94080192e-01]
 [9.99956489e-01]
 [2.26443131e-02]
 [9.96342599e-01]
 [9.33357894e-01]
 [9.99888062e-01]
 [9.92547333e-01]
 [4.81348373e-02]
 [4.19849250e-03]
 [9.99988556e-01]
 [9.99686360e-01]
 [9.98901606e-01]
 [3.37215103e-02]
 [4.61399287e-01]
 [2.71743257e-02]
 [1.24786478e-02]
 [9.98506963e-01]
 [1.00000000e+00]
 [2.50740815e-02]
 [1.41607793e-02]
 [1.45521635e-05]
 [5.23481190e-01]
 [1.00000000e+00]
 [9.12033319e-01]
 [1.88986249e-02]
 [9.98307586e-01]
 [9.99685526e-01]
 [9.64697152e-02]
 [7.85798192e-01]
 [4.03584214e-03]
 [3.61067861e-01]
 [4.57662567e-02]
 [3.73947881e-02]
 [3.226513

In [55]:
from decimal import Decimal   
index_j =[] 
for e in j_pre:
    if e[0] > 0.9:
        #print(e)
        index_j.append(j_pre.tolist().index(e))
print(len(j_song))
print(len(index_j))
for e in index_j:
    print(j_song[e])

121
50
Come Around Me
Yummy
Take It Out On Me
Second Emotion (feat. Travis Scott)
I'll Show You
What Do You Mean?
Sorry
Love Yourself
No Pressure
The Feeling
Life Is Worth Living
Where Are Ü Now
Children
Get Used To It
We Are
Trust
Hold Tight
I'll Show You
I'll Show You
Confident
Backpack
What's Hatnin'
All Around The World
As Long As You Love Me
Take You
Fall
Die In Your Arms
Thought Of You
Beauty And A Beat
One Love
Out Of Town Girl
She Don't Like The Lights
Maria
Never Say Never
Up
Overboard (Live)
Only Thing I Ever Get For Christmas
Fa La La
All I Want For Christmas Is You (SuperFestive!) Duet with Mariah Carey
Home This Christmas
All I Want For Christmas Is You (SuperFestive!) Duet with Mariah Carey
Pray
Baby
Stuck In The Moment
U Smile
Never Let You Go
One Time
Favorite Girl
Love Me
Where Are You Now?


In [56]:
b_pre = model.predict(np.asarray(bf))
index_b =[] 
for e in b_pre:
    if e[0] > 0.9:
        #print(e)
        index_b.append(b_pre.tolist().index(e))
print(len(b_song))
print(len(index_b))
for e in index_b:
    print(b_song[e])

446
35
She Came In Through The Bathroom Window - 2019 Mix
The End - 2019 Mix
The Ballad Of John And Yoko - Take 7
Octopus's Garden - Take 9
Come Together - Take 5
The End - Take 3
Rocky Raccoon - Esher Demo
Boys - Live / Remastered
She Loves You - Live / Remastered
Long Tall Sally - Live / Remastered
Baby You're A Rich Man
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Ticket To Ride - Remastered 2015
Get Back - Remastered 2015
Come Together - Take 5
Maggie Mae - Remastered 2009
She Came In Through The Bathroom Window - Remastered 2009
Back In The U.S.S.R. - Remastered 2009
Savoy Truffle - Remastered 2009
Come Together - Take 5
Baby, You're A Rich Man - Remastered 2009
Fixing A Hole - Remix
When I'm Sixty-Four - Take 2
Good Morning Good Morning - Take 8
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Come Together - Take 5
Tell Me What You See - Remaste

In [53]:
print(bet_pre.tolist())


[[1.2134953976783436e-05], [8.26910309115192e-06], [0.2147088497877121], [0.00018855620874091983], [0.004757057409733534], [8.001082278497051e-06], [0.011700285598635674], [0.002573251025751233], [3.5337406334434718e-09], [1.6208508668569266e-07], [0.05483655259013176], [0.0007019756594672799], [0.9999988079071045], [2.4925789148255717e-06], [1.636165688978508e-05], [0.9999969005584717], [2.5353515411552507e-06], [1.2721550035621476e-07], [0.046365007758140564], [0.04862150922417641], [0.9998482465744019], [6.583393405890092e-05], [0.025758860632777214], [0.9930896162986755], [0.002191532403230667], [3.081652710790195e-10], [0.005200669635087252], [0.000507501361425966], [1.7759572074282914e-05], [1.0], [0.9999939203262329], [3.4561471693450585e-06], [1.2643204172491096e-06], [6.112151366206609e-17], [0.004589435178786516], [0.00018855441885534674], [1.537937168905046e-05], [0.014429558999836445], [0.0], [5.523364968521016e-13], [0.30008310079574585], [0.00106070248875767], [3.86125066

In [54]:
print(j_pre.tolist())

[[3.5791472328128293e-06], [0.09698155522346497], [0.9934764504432678], [0.058716002851724625], [0.9998425245285034], [0.016176844015717506], [0.0143485302105546], [0.07688658684492111], [0.9999301433563232], [0.9999710321426392], [0.1153898537158966], [0.005018068011850119], [0.017520347610116005], [0.00010230121552012861], [0.013330414891242981], [0.02742741070687771], [0.6213701367378235], [4.280880798500711e-15], [1.0], [0.9695936441421509], [0.9997403025627136], [0.9985767602920532], [0.39408019185066223], [0.999956488609314], [0.02264431305229664], [0.9963425993919373], [0.9333578944206238], [0.9998880624771118], [0.992547333240509], [0.04813483729958534], [0.004198492504656315], [0.9999885559082031], [0.9996863603591919], [0.9989016056060791], [0.03372151032090187], [0.46139928698539734], [0.0271743256598711], [0.012478647753596306], [0.9985069632530212], [1.0], [0.025074081495404243], [0.014160779304802418], [1.4552163520420436e-05], [0.5234811902046204], [1.0], [0.912033319473